In [1]:
import numpy
from sklearn.metrics import accuracy_score
import tensorflow as tf
import matplotlib.pyplot as plt

from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.shapelets import LearningShapelets, \
    grabocka_params_to_shapelet_size_dict
from tslearn.utils import ts_size
import numpy
from sklearn.metrics import accuracy_score

from tslearn.generators import random_walk_blobs
from tslearn.preprocessing import TimeSeriesScalerMinMax, \
    TimeSeriesScalerMeanVariance
from tslearn.neighbors import KNeighborsTimeSeriesClassifier, \
    KNeighborsTimeSeries
import pandas as pd
import numpy as np
import random
from tslearn.datasets import UCR_UEA_datasets
import sktime
from tslearn.preprocessing import TimeSeriesResampler
import random
from sktime.classification.kernel_based import RocketClassifier
from sktime.datasets import load_unit_test

In [4]:
from sktime.classification.kernel_based import RocketClassifier
from sktime.datasets import load_unit_test
X_train, y_train = load_unit_test(split="train", return_X_y=True)
X_test, y_test = load_unit_test(split="test", return_X_y=True) 
clf = RocketClassifier(num_kernels=500) 
clf.fit(X_train, y_train) 
y_pred = clf.predict(X_test) 

In [18]:
df = pd.read_pickle("Train.pkl")
randomlist1 = random.sample(range(0, 260), 78)

df_test1 = df.loc[df["Batch"].isin(randomlist1)]
df_train1 = df.loc[~df["Batch"].isin(randomlist1)]
df_steps1 = df_test1.loc[df_test1["CuStepNo ValueY"] != df_test1["Next_Step"]]
df_steps2 = df_train1.loc[df_train1["CuStepNo ValueY"] != df_train1["Next_Step"]]

In [19]:
y_train1 = df_steps2["DeviationID ValueY"].astype(int)
y_test1 = df_steps1["DeviationID ValueY"].astype(int)
X_train1 = df_train1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)
X_test1 = df_test1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)

In [26]:
X_train_step1 = []
for i in X_train1.Batch.unique():
        for j in X_train1["CuStepNo ValueY"].unique():
                num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)&(X_train1["CuStepNo ValueY"] == j)].drop(["Batch"],axis=1))
                X_train_step1.append(num)

In [27]:
X_test_step1 = []
for i in X_test1.Batch.unique():
        for j in X_test1["CuStepNo ValueY"].unique():
                num = numpy.array(X_test1.loc[(X_test1["Batch"] == i)&(X_test1["CuStepNo ValueY"] == j)].drop(["Batch"],axis=1))
                X_test_step1.append(num)

In [14]:
length = []
for i in X_train_step1:
    length.append(len(i))

In [28]:
X_train_s1 = TimeSeriesResampler(sz=150).fit_transform(X_train_step1)
X_test_s1 = TimeSeriesResampler(sz=150).fit_transform(X_test_step1)

In [25]:
y_train1

2102      5
2350      5
2416      5
3012      5
3212      1
         ..
264688    9
264886    9
265190    9
265261    9
265798    9
Name: DeviationID ValueY, Length: 696, dtype: int32

In [30]:
X_test_s1.shape

(308, 150, 40)

In [40]:
X_train = X_train_s1.reshape(696,40,150)

In [41]:
X_test = X_test_s1.reshape(308,40,150)

In [37]:
scaler = TimeSeriesScalerMinMax()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
scaler = TimeSeriesScalerMinMax()
X_train = scaler.fit_transform(X_train_s1)
X_test = scaler.transform(X_test_s1)

In [42]:
clf = RocketClassifier(num_kernels=500) 
clf.fit(X_train, y_train1) 
y_pred = clf.predict(X_test) 

In [45]:
clf.estimator_

SklearnClassifierPipeline(classifier=RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03])),
                          transformers=[Rocket(num_kernels=500),
                                        TabularToSeriesAdaptor(transformer=StandardScaler(with_mean=False))])

In [43]:
accuracy_score(y_test1, y_pred)

0.8376623376623377

In [20]:
y_pred

array([ 1,  2,  7,  1,  1,  1, 10,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2, 10, 10,  9,  4,  1,  9,  9, 10,  4,  3,  3,
        9, 10,  4,  3,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,
        6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  2,
        9,  9,  9, 10,  3,  9,  1,  1])